In [1]:
%%capture
!pip install pytesseract
!pip install tqdm
!pip install joblib
!pip install pandas
!pip install numpy
!pip install opencv-python
!apt-get install tesseract-ocr -y

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import pytesseract
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import imghdr
import re
import joblib
import datetime
import json

In [3]:
from google.colab import drive

# Correct mount point without spaces
drive.mount('/content/Mon_drive', force_remount=True)

Mounted at /content/Mon_drive


In [10]:
class Config:
    # Chemins principaux (utilisez DriveConfig.BASE_PATH si vous êtes sur Google Drive)
    BASE_PATH = "/content/Mon_drive/MyDrive/Projet_ml"
    DATA_PATH = f"{BASE_PATH}/data/Bing_images"
    MODELS_PATH = f"{BASE_PATH}/models"
    OUTPUT_PATH = f"{BASE_PATH}/output"

    # Paramètres de validation d'image
    VALID_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.pdf'}
    MIN_IMAGE_SIZE = (100, 100)
    MAX_FILE_SIZE = 10 * 1024 * 1024  # 10MB

In [28]:
def check_drive_mounted():
        """Vérifie si Google Drive est monté"""
        return os.path.exists("/content/Mon_drive")

def count_files_by_sector():
    """Compte les fichiers par secteur"""
    counts = {}
    if not os.path.exists(Config.DATA_PATH):
        print("Dossier data non trouvé!")
        return counts

    for sector in os.listdir(Config.DATA_PATH):
        sector_path = os.path.join(Config.DATA_PATH, sector)
        if os.path.isdir(sector_path):
            counts[sector] = len([f for f in os.listdir(sector_path)
                                if os.path.isfile(os.path.join(sector_path, f))])

    return counts

In [12]:
check_drive_mounted()
count_files_by_sector()

{'SAP Developer resumes': 64,
 'Testing resumes': 76,
 'Sales resumes': 95,
 'SQL Developer resumes': 82,
 'web designing resumes': 59,
 'React Developer resumes': 18,
 'Managment resumes': 70,
 'Operations Manager resumes': 79,
 'Public Relations resumes': 71,
 'HR resumes': 88,
 'Java Developer resumes': 78,
 'Mechanical Engineer resumes': 105,
 'PMO resumes': 52,
 'Python Developer resumes': 58,
 'Information Technology resumes': 50,
 'Network Security Engineer resumes': 64,
 'Health_Fitness resumes': 77,
 'Digital Media resumes': 68,
 'Designing resumes': 39,
 'Electrical Engineering resumes': 79,
 'ETL Developer resumes': 65,
 'Education resumes': 62,
 'DotNet Developer resumes': 63,
 'DevOps Engineer resumes': 87,
 'Finance resumes': 58,
 'Food_Beverages resumes': 62,
 'data science resumes': 120,
 'Consultant resumes': 73,
 'Blockchain resumes': 9,
 'Database resumes': 76,
 'Banking resumes': 43,
 'Building _Construction resumes': 56,
 'BPO resumes': 30,
 'Business Analyst resum

In [13]:
def is_valid_image(file_path):
        """Vérifie si l'image est valide"""
        try:
            ext = os.path.splitext(file_path)[1].lower()
            if ext not in Config.VALID_EXTENSIONS:
                return False, "Extension non valide"

            if os.path.getsize(file_path) > Config.MAX_FILE_SIZE:
                return False, "Fichier trop volumineux"

            img = cv2.imread(file_path)
            if img is None:
                return False, "Fichier non lisible comme image"

            height, width = img.shape[:2]
            if height < Config.MIN_IMAGE_SIZE[0] or width < Config.MIN_IMAGE_SIZE[1]:
                return False, "Image trop petite"

            return True, "Image valide"
        except Exception as e:
            return False, f"Erreur: {str(e)}"

In [18]:
def preprocess(image):
        """Prétraite l'image pour améliorer l'OCR"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        denoised = cv2.fastNlMeansDenoising(gray)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        enhanced = clahe.apply(denoised)
        return enhanced
def extract_text(image):
        """Extrait le texte de l'image"""
        try:
            text = pytesseract.image_to_string(image)
            return text.strip()
        except Exception as e:
            print(f"Erreur lors de l'extraction du texte: {str(e)}")
            return ""

In [25]:
def process_and_save_data(base_path, output_csv_path):
    """Traite les données et les sauvegarde en CSV"""
    data = {
        'Category': [],
        'Resume': []
    }

    stats = {'total': 0, 'valid': 0, 'invalid': 0, 'errors': {}}

    for sector in os.listdir(base_path):
        sector_path = os.path.join(base_path, sector)
        sector.replace("resumes", "").strip()
        if os.path.isdir(sector_path):
            print(f"\nTraitement du secteur: {sector}")

            for cv_file in tqdm(os.listdir(sector_path)):
                stats['total'] += 1
                cv_path = os.path.join(sector_path, cv_file)

                try:
                    is_valid, message = is_valid_image(cv_path)

                    if not is_valid:
                        data['Category'].append(sector)
                        data['Resume'].append('')
                        stats['invalid'] += 1
                        stats['errors'][cv_path] = message
                        continue

                    image = cv2.imread(cv_path)
                    processed_image = preprocess(image)
                    text = extract_text(processed_image)

                    if len(text) > 50:
                        status = "Valid"
                        stats['valid'] += 1
                    else:
                        status = "Invalid: Texte insuffisant"
                        stats['invalid'] += 1
                        stats['errors'][cv_path] = "Texte insuffisant"

                    data['Category'].append(sector)
                    data['Resume'].append(text)

                except Exception as e:
                    stats['invalid'] += 1
                    stats['errors'][cv_path] = str(e)

                    data['Category'].append(sector)
                    data['Resume'].append('')

    df = pd.DataFrame(data)
    df.to_csv(output_csv_path, index=False)

    print("\nStatistiques du traitement:")
    print(f"Total des fichiers: {stats['total']}")
    print(f"Fichiers valides: {stats['valid']}")
    print(f"Fichiers invalides: {stats['invalid']}")

    return df, stats

In [27]:
df, stats = process_and_save_data(Config.DATA_PATH,os.path.join(Config.OUTPUT_PATH, "dataset_resumes.csv"))


Traitement du secteur: SAP Developer resumes


100%|██████████| 64/64 [07:24<00:00,  6.94s/it]



Traitement du secteur: Testing resumes


100%|██████████| 76/76 [10:03<00:00,  7.94s/it]



Traitement du secteur: Sales resumes


...


Traitement du secteur: Advocate resumes


100%|██████████| 94/94 [10:25<00:00,  6.65s/it]



Traitement du secteur: Accountant resumes


100%|██████████| 67/67 [08:40<00:00,  7.77s/it]



Statistiques du traitement:
Total des fichiers: 2852
Fichiers valides: 2724
Fichiers invalides: 128
